# MICrONS Neuron Mesh Download

In [1]:
from caveclient import CAVEclient
from meshparty import trimesh_io, trimesh_vtk
import cloudvolume 
import numpy as np

The following assumes you are running in a local environment, and have already set a cave token. If not, see _ProgrammaticAccess.ipynb_ to do so.

In [2]:
# Inititalize the CAVEclient
client = CAVEclient('minnie65_public', auth_token='a549b45f5f69c48cf6becaba42392595')

# specify the materialization version, for consistency across time",
client.version = 1300

__Versioning note:__ We use version 1300 is this notebook because it corresponds with the most recent static segmentation, which is what you see on [microns-explorer.org](https://www.microns-explorer.org/gallery-mm3)

The most recent public version at the time of writing (9/30/2025) is __version 1507__.

## What is a Mesh?
A mesh is a set of vertices connected via triangular faces to form a 3 dimensional representation of the outer membrane of a neuron, glia or nucleus.

### Meshes can either be static or dynamic:
##### Static:
- pros: smaller files thus easier to work with, multiple levels of detail (lod) which can be accessed (example below)
- cons: may include false gaps and merges from self contacts, updated less frequently

##### Dynamic:
- pros: highly detailed thus more reflective of biological reality and backed by proofreading infrastructure CAVE (Connectome Annotation Versioning Engine)
- cons: much larger files, only one level of detail

In [3]:
# to access dynamic meshes, you can query the segmentation source from the info client
client.info.segmentation_source()

'graphene://https://minnie.microns-daf.com/segmentation/table/minnie65_public'

In [4]:
# this can be used to initialize a cloudvolume object
cv = cloudvolume.CloudVolume(client.info.segmentation_source(), progress=False, use_https=True)

Given a root_id, __CloudVolume__ can be used to retrieve the mesh. 
`cloudvolume.mesh.get()` returns a dictionary with the neuron segment id as the key and the mesh as the value

In [5]:
# Example: pyramidal cell in v1300
example_cell_id = 864691135572530981
%time mesh = cv.mesh.get(example_cell_id)[example_cell_id]

CPU times: total: 53.3 s
Wall time: 1min 5s


The returned mesh includes:
* vertices : Nx3 [x, y, z] positions in nm
* faces: Kx3 [i, j, k] indices into vertices that describe the triangular meshes

In [6]:
print(mesh.vertices.shape, mesh.faces.shape)

(4390777, 3) (8761561, 3)


Since downloading meshes can take some time, particularly for these dynamic meshes,
it is convient to cache them on disk.     
     
To facilitate the analysis of meshes, we developed a package called __MeshParty__ that we will use here to enable a cache.

In [7]:
# to enable a cache, create a MeshMeta object
mm = trimesh_io.MeshMeta(cv_path = client.info.segmentation_source(),
                         disk_cache_path='minnie65_meshes',
                         map_gs_to_https=True)

You can get a mesh like this and it will be cached in memory and in disk in case you need it again.    
Restart the kernel and run the below cells again to see the difference.   

You'll find the mesh file saved as an hdf5 file in the "minnie65_meshes"
subdirectory

In [8]:
mesh = mm.mesh(seg_id=example_cell_id)

The MeshParty object has more useful properties and attributes
such as:
* a scipy.csgraph sparse graph object (mesh.csgraph)
*  a networkx graph object (mesh.nxgraph) 

Read more about what you can do with MeshParty on its [Documentation](https://meshparty.readthedocs.io/en/latest/?badge=latest).

In particular it lets you associate _skeletons_, and _annotations_ onto the mesh into a "meshwork" object. 

## Static Meshes    
The meshes that are available in the visualization on microns-explorer.org are faster because they are static and have been downsampled multiple times. However, this comes with the drawback of being less biologically accurate.

Level of detail is controlled with optional argument, where `lod=0` is the highest level of detail

we can access one of these downsampled static meshes by setting the path here:

In [7]:
## Previous versions of this tutorial have used v117 meshes, with the following path

cv = cloudvolume.CloudVolume("precomputed://gs://iarpa_microns/minnie/minnie65/seg_m1300", use_https=True)

In [8]:
# the cloud volume interface is the same but it is a faster initial download 
%time mesh = cv.mesh.get(example_cell_id, lod=3)[example_cell_id]

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.49s/it]

CPU times: total: 62.5 ms
Wall time: 1.51 s


In [9]:
# as you can see the meshes aren't exactly the same as before. They because they have not been downsampled
mesh.vertices.shape, mesh.faces.shape

((56938, 3), (99936, 3))

In addition, the Static meshes are available in 3 levels of detail, this covers two orders of magnitude of detail
which is what neuroglancer leverages to efficiently load the data at the resolution necessary to render the current scene.  

In [10]:
for lod in range(4):
    mesh = mesh = cv.mesh.get(example_cell_id, lod=lod)[example_cell_id]
    print(f"level of detail {lod}: n_verts: {mesh.vertices.shape[0]} n_faces: {mesh.faces.shape[0]}")

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.92s/it]


level of detail 0: n_verts: 5231141 n_faces: 10291430


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.67s/it]


level of detail 1: n_verts: 1457607 n_faces: 2828410


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.26s/it]


level of detail 2: n_verts: 253583 n_faces: 474769


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.16it/s]

level of detail 3: n_verts: 56938 n_faces: 99936


Now you have the neuron mesh, at your chosen level of detail, downloaded and ready to use locally. For next steps, consider:

1. Mesh visualization and rendering with a general use package like **PyVista**, see example here https://tutorial.microns-explorer.org/examples/astrocyte_meshes.html 
2. Mesh repair and visualization [examples with __meshparty__](https://meshparty.readthedocs.io/en/latest/guide/visualization.html) 
3. Mesh simplification and analysis [from our friends at __Navis__](https://navis-org.github.io/navis/generated/gallery/4_remote/tutorial_remote_02_microns/)
4. Exporting the meshes into Neuron, or simulation program of your choice